<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
%%capture
import ROOT
import glob
import math
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
from IPython.display import display, HTML
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *

In [ ]:
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

In [ ]:
# Read the Root Files
path = "/cave/alice/data/2022/LHC22o/apass2/"
fileList = glob.glob(path+"*_QC.root")
fileList.sort()
#fileList = fileList[13:]
runList = [fileList[i][36:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))

In [ ]:
qualities = pd.DataFrame({'runNumber':range(10),'hist1': np.random.randint(3, size=10), 'hist2': np.random.randint(3, size=10)})
qualities

In [ ]:
%jsroot on
ROOT.gROOT.ForceStyle()
ROOT.gStyle.SetPalette(4)
ROOT.gStyle.SetGridStyle(1);

qualityLabels = {0:"NULL",1:"GOOD",2:"MEDIUM",3:"BAD"}

canvas = ROOT.TCanvas()
canvas.SetLeftMargin(0.15);
canvas.SetBottomMargin(0.15);
canvas.SetGrid()

qualityHist = ROOT.TH2I("QualityMatrix","Histogram showing qualities from checks",
                        len(qualities.index),min(qualities.index),max(qualities.index)+1,
                        len(qualities.columns)-1,0,len(qualities.columns)-1)
qualityHist.SetCanExtend(ROOT.TH1.kAllAxes)
qualityHist.SetStats(0)
for runIndex,run in enumerate(qualities.runNumber):
    qualityHist.GetXaxis().SetBinLabel(runIndex+1,str(run))
    qualityHist.GetXaxis().SetTickLength( 0.03)
    for checkIndex,check in enumerate(qualities.loc[:, qualities.columns != "runNumber"].columns):
        #print(checkIndex,runIndex)
        qualityHist.Fill(run,check,qualities.iloc[runIndex][check])

qualityHist.LabelsOption("u")
qualityHist.Draw("colz")

canvas.Draw()

In [ ]:
qualities[qualities["runNumber"]== 1]

In [ ]:
%jsroot on
objectName="hPt;1"
trending = "entries"
error = "stdDev"

[trend,canvas] = drawTrending(objectName,rootDataFile,files=22,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1)
[qualities,canvas] = checkTrending(trend,canvas=canvas)

canvas.Draw()